## Purpose

- Goal
- Refresher on spatial random field models
- Extensions in space-time
    - IID spatiotemporal random field
    - random walk spatiotemporal random field
    - AR(1) spatiotemporal random field
- Code demonstration for the AR(1) st model 


# A quick refresher 

## We have now seen temporal models 

::: {style="font-size:34px"}

$$
\begin{array}{c}
z_{t}= \alpha + z_{t-1}+\varepsilon_{t}, \quad \varepsilon_{t} \sim \mathrm{N}\left(0, \sigma_{p}^{2}\right), & \color{#E78021}{\text{[process equation]}} \\\\
y_{t}= z_{t}+\eta_{t}, \quad \eta_{t} \sim \mathrm{N}\left(0, \sigma_{o}^{2}\right) . & \color{#8D44AD}{\text{[observation equation]}} \\
\end{array}
$$

- The time-series follows a random walk with drift or trend term $\alpha$
- $\sigma_{p}^{2}$ represents process variance
- $\sigma_{o}^{2}$ represents obsrvation variance
- $\varepsilon_{t}$ represent process errors 
- $\eta_{t}$ represent observation errors

:::

[Auger-Methe et al. 2021]{.footerRight}

## And we have also seen spatial models 

$$
\begin{array}{l}\epsilon_{s} \sim \operatorname{MVN}(0, \Sigma)\\
\text{where } \Sigma \text{ is a covariance matrix with:}\\
\Sigma_{i, j}=\sigma^{2}_{gp} \exp \left( - \text {distances}_{i,j} / \theta_{gp}\right), \text { if } i \neq j \\
\Sigma_{i, j}=\sigma^{2}_{gp}, \text { if } i=j\end{array}
$$

::: {style="font-size:29px"}

- $\epsilon_{s}$ is a spatially explicit random effect
- $\theta_{gp}$ controls how quickly the correlation decays between locations
- $\sigma^{2}_{gp}$ is the marginal variability of the spatial function at all locations
- $\text {distances}_{i,j}$ is a matrix describing Euclidean distance between sample locations 
:::

[Cressie and Wikle 2011; Anderson and Ward 2019]{.footerRight}

## The problem

- What if our data are indexed not only in space, but also in time? 

[Cressie and Wikle 2011; Anderson and Ward 2019]{.footerRight}

## The problem

- What if our data are indexed not only in space, but also in time? 
- We might need to account for this additional dependency in our data

[Cressie and Wikle 2011; Anderson and Ward 2019]{.footerRight}

## The problem

- What if our data are indexed not only in space, but also in time? 
- We might need to account for this additional dependency in our data
    - Why should we care about dependency?

[Hurlburt 1984; Cressie and Wikle 2011; Anderson and Ward 2019]{.footerRight}

## The problem

- What if our data are indexed not only in space, but also in time? 
- We might need to account for this additional dependency in our data
    - Why should we care about dependency?
- More generally, why go to all of this effort with spatiotemporal data?

[Hurlburt 1984; Cressie and Wikle 2011; Anderson and Ward 2019]{.footerRight}

# Spatiotemporal random fields

[[see also this helpful link from the sdmTMB documentation](https://pbs-assess.github.io/sdmTMB/articles/model-description.html)]{.footerRight}


## IID spatiotemporal random fields

- This is perhaps (?) the simplest spatio-temporal random field

$$
\varepsilon_{s,t} \sim \operatorname{MVN}\left(\mathbf{0}, \Sigma\right)
$$

## IID spatiotemporal random fields

- This is perhaps (?) the simplest spatio-temporal random field

$$
\varepsilon_{s,t} \sim \operatorname{MVN}\left(\mathbf{0}, \Sigma\right)
$$

- Think of $\varepsilon_{s,t}$ as a matrix of sites by years 

## IID spatiotemporal random fields

- This is perhaps (?) the simplest spatio-temporal random field

$$
\varepsilon_{s,t} \sim \operatorname{MVN}\left(\mathbf{0}, \Sigma\right)
$$

- Think of $\varepsilon_{s,t}$ as a matrix of sites by years 

## How do we get $\Sigma$?

$$
\begin{array}{l} \Sigma \text{ is a covariance matrix with:}\\
\Sigma_{i, j}=\sigma^{2}_{gp} \exp \left( - \text {distances}_{i,j} / \theta_{gp}\right), \text { if } i \neq j \\
\Sigma_{i, j}=\sigma^{2}_{gp}, \text { if } i=j\end{array}
$$

## How do we get $\Sigma$?

$$
\begin{array}{l} \Sigma \text{ is a covariance matrix with:}\\
\Sigma_{i, j}=\sigma^{2}_{gp} \exp \left( - \text {distances}_{i,j} / \theta_{gp}\right), \text { if } i \neq j \\
\Sigma_{i, j}=\sigma^{2}_{gp}, \text { if } i=j\end{array}
$$

- How else could we get $\Sigma$?

## Let's simulate that spatiotemporal field in Stan

- simulate an iid spatio-temporal random field with an exponential kernel 


```{javascript echo = T, eval = F}
data {
  int<lower=1> n_sites;
  int<lower=1> n_year;
  matrix[n_sites, n_sites] dist_sites;
  real<lower=0> gp_theta;
  real<lower=0> gp_sigma;
}
generated quantities {
  vector[n_sites] mu_zeros;
  matrix[n_sites, n_sites] SIGMA;
  real<lower=0> gp_sigma_sq;
  matrix[n_sites, n_year] eps_st;  
  // transformations
  gp_sigma_sq = pow(gp_sigma, 2.0);
  // cov matrix between sites
  SIGMA = gp_sigma_sq * exp(-dist_sites / gp_theta);
  for (k in 1:n_sites) {
    mu_zeros[k] = 0;
  }
  for(t in 1:n_year){
    eps_st[,t] = multi_normal_rng(mu_zeros, SIGMA);
  }
}


```


## References

::: {style="font-size:29px"}
- Anderson and Ward. 2019.  Black swans in space: modeling spatiotemporal processes with extremes. Ecology.

-  Auger-Methe et al. 2021. A guide to state-space modeling of ecological time series. Ecological Monographs.

- Cressie and Wikle 2011.  Statistics for spatio-temporal data. 

- Hurlburt 1984. Pseudoreplication and the Design of Ecological Field Experiments. Ecological Monographs.

:::